## This Extractive Text Summarisation was based on the links below:
* https://www.mygreatlearning.com/blog/text-summarization-in-python/#Approaches%20used%20for%20Text%20Summarization

### Import Relevant Libraries

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import pandas as pd
import numpy as np

C:\Users\dianfarahr.2019\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


### Load Dataset

In [4]:
contracts = pd.read_excel('../data/contract_new.xlsx')
contracts['content'] = contracts['content'].apply(lambda x: x.lower())
contracts.head()

,contract,content
0,2ThemartComInc_19990826_10-12G_EX-10.10_670028...,co-branding and advertising agreement this co-...
1,ABILITYINC_06_15_2020-EX-4.25-SERVICESAGREEMEN...,exhibit 4.25 information in this exhibit ident...
2,ACCELERATEDTECHNOLOGIESHOLDINGCORP_04_24_2003-...,exhibit 10.13 joint venture agreement collecti...
3,ACCURAYINC_09_01_2010-EX-10.31-DISTRIBUTORAGRE...,exhibit 10.31 pursuant to 17 c.f.r. § 240.24b-...
4,ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENTAG...,redacted copy confidential treatment requested...


### Summarize using Avg Sentence Score

In [8]:
#get an input text to do a rough test
input_text = contracts['content'][1]
print(input_text)

exhibit 4.25 information in this exhibit identified by [ * * * ] is confidential and has been excluded because it is both (i) not material and (ii) would likely cause competitive harm to the registrant if publicly disclosed. services agreement this services agreement (this "agreement") is entered into on october 1, 2019 and is made effective as of november 1, 2019 (the "effective date"), by and between [ * * * ] (the "provider"), and telcostar pte, ltd., a company organized and existing under the laws of singapore and ability computer & software industries ltd, a company organized and existing under the laws of the state of israel (each and both of them "recipient"). each of the foregoing parties is referred to herein as a "party" and together as the "parties". recitals a. recipient wishes to engage the provider to provide certain services and resources (the "services") and provider desires to provide recipient with the services all in accordance with the terms and conditions set forth

In [9]:
#tokenise the text
stopWords = set(stopwords.words("english"))
words = word_tokenize(input_text)

In [10]:
#create a freq table to keep the score of each word
freqTable = dict()
for word in words:
    if word in stopWords:
        continue
    if word in freqTable:
        freqTable[word] += 1
    else:
        freqTable[word] = 1

In [14]:
# Create dictionary to keep the score of each sentence
sentences = sent_tokenize(input_text)
sentenceValue = dict()
for sentence in sentences:
    for word, freq in freqTable.items():
        if word in sentence:
            if sentence in sentenceValue:
                sentenceValue[sentence] += freq
            else:
                sentenceValue[sentence] = freq
                
sumValue = 0
for sentence in sentenceValue:
    sumValue += sentenceValue[sentence]

In [15]:
# Average value of a sentence from the original text
avg = int(sumValue/len(sentenceValue))

In [17]:
# Storing sentences into our summary
summary = ''
summary_list = []
for sentence in sentences:
    if (sentence in sentenceValue) and (sentenceValue[sentence]>(1.2*avg)):
        summary_list.append(sentence)
        summary += ' '+ sentence
        
print(summary)

 services agreement this services agreement (this "agreement") is entered into on october 1, 2019 and is made effective as of november 1, 2019 (the "effective date"), by and between [ * * * ] (the "provider"), and telcostar pte, ltd., a company organized and existing under the laws of singapore and ability computer & software industries ltd, a company organized and existing under the laws of the state of israel (each and both of them "recipient"). (a) provider agrees to provide the services set forth on the exhibit a attached hereto (as such exhibit may be amended or supplemented pursuant to the terms of this agreement, the "exhibit") to recipient for the respective periods and on the other terms and conditions set forth in this agreement and in the exhibit. notwithstanding the contents of the exhibit, provider agrees to respond in good faith to any reasonable request by recipient for access to any additional services and resources that are necessary for the operation of the recipient 

In [19]:
print(len(sentences))
print(len(summary_list))

121
54
